# Debates in the Oireachtas: 02 - Dataframes


In [3]:
import os
import sys
import uuid
import json
import lzma
import requests
from glob import glob
from datetime import datetime
from multiprocessing import Pool
import pandas as pd
from pandas.io.json import json_normalize

#import tqdm
from tqdm import tqdm_notebook

# Max results available for paging through with limit & skip is 10,000 so splitting by years works best
def save_page_year(page, year, overwrite=False):
    fname = "../data/{}/{}.json.xz".format(page,year)
    url = "https://api.oireachtas.ie/v1/{}?date_start={}&date_end={}&limit=10000".format(page, year, year+1)
    cache_url(url, fname)

# Save a given url as a file
def cache_url(url, fname, overwrite=False):
    if os.path.isfile(fname) and not overwrite:
        return

    data = requests.get(url).content
    with lzma.open(fname, 'wb') as f:            
        f.write(data)

# Show Progress bar and perform a function on an iterable in parallel:
def process_with_progress(func, iterable):
    with Pool(8) as p:
        results = list(tqdm_notebook(p.imap(func, iterable), total=len(iterable)))
    return results

# Preprocess:

# Flatten json / dict values in a column
def flatten_column(df, column):
    tmp_df = json_normalize([r for r in df[column]])
    tmp_df.rename(columns=lambda k: column+'.'+k, inplace=True)
    tmp_df.index = df.index
    return tmp_df

def flatten_dataframe_columns(df, columns):
    df_expanded = pd.concat([flatten_column(df, column) for column in columns], axis=1)
    df.drop(columns, axis=1, inplace=True) # remove flattened originally json columns
    df = pd.concat([df_expanded, df], axis=1)
    return df

In [4]:
def save_debates_page(year, overwrite=False):
    save_page_year('debates', year, overwrite)
    
    
# Entire dataset: 19349 as of 15-12-2017, starts at 1919
years = range(1919, datetime.now().year + 1)
print("Debates Pages:")
process_with_progress(save_debates_page, years)

# Update this year
save_debates_page(datetime.now().year, True)

Debates Pages:


In [5]:
with lzma.open('../data/debates/2019.json.xz', 'rb') as f:
    preview = json.loads(f.read().decode('utf-8'))

JSONDecodeError: Expecting value: line 1 column 3433913 (char 3433912)

In [7]:
RenderJSON(preview)

In [ ]:
#preview

In [9]:
# Load Data from a single file and return dataframe
def debates_df(fname):
    with lzma.open(fname, 'rb') as f:
        data = json.loads(f.read().decode('utf-8'))
        #print(fname, len(data['results']))
        
        # We don't care about the "head" part for now, just "results"
        records = json.dumps([r['debateRecord'] for r in data['results']])
        df = pd.read_json(records, orient='records', dtype=False)
        return df

In [14]:
debates_df("data/year/2007.json.xz")

,date,debateSections,lastUpdated,debateType,formats,chamber,counts,uri,house
0,2007-12-20,"[{'debateSection': {'debateType': 'prelude', '...",2017-03-24T19:42:28+00:00,debate,"{'pdf': None, 'xml': {'uri': '/akn/ie/debateRe...","{'showAs': 'Seanad Éireann', 'uri': '/ie/oirea...","{'questionCount': 0, 'billCount': 5, 'contribu...",/akn/ie/debateRecord/seanad/2007-12-20/debate/...,"{'showAs': '23rd Seanad', 'committeeCode': '',..."
1,2007-12-19,"[{'debateSection': {'debateType': 'prelude', '...",2017-03-24T19:28:12+00:00,debate,"{'pdf': None, 'xml': {'uri': '/akn/ie/debateRe...","{'showAs': 'Dáil Éireann', 'uri': '/ie/oireach...","{'questionCount': 380, 'billCount': 1, 'contri...",/akn/ie/debateRecord/dail/2007-12-19/debate/main,"{'showAs': '30th Dáil', 'committeeCode': '', '..."
2,2007-12-19,"[{'debateSection': {'debateType': 'prelude', '...",2017-03-24T18:29:06+00:00,debate,"{'pdf': None, 'xml': {'uri': '/akn/ie/debateRe...","{'showAs': 'JOINT COMMITTEE ON ENTERPRISE, TRA...","{'questionCount': 0, 'billCount': 0, 'contribu...",/akn/ie/debateRecord/joint_committee_on_enterp...,"{'showAs': 'JOINT COMMITTEE ON ENTERPRISE, TRA..."
3,2007-12-19,"[{'debateSection': {'debateType': 'prelude', '...",2017-03-24T18:32:56+00:00,debate,"{'pdf': None, 'xml': {'uri': '/akn/ie/debateRe...","{'showAs': 'SELECT COMMITTEE ON ENTERPRISE, TR...","{'questionCount': 0, 'billCount': 1, 'contribu...",/akn/ie/debateRecord/select_committee_on_enter...,"{'showAs': 'SELECT COMMITTEE ON ENTERPRISE, TR..."
4,2007-12-19,"[{'debateSection': {'debateType': 'prelude', '...",2017-03-24T18:30:42+00:00,debate,"{'pdf': None, 'xml': {'uri': '/akn/ie/debateRe...",{'showAs': 'JOINT COMMITTEE ON FOREIGN AFFAIRS...,"{'questionCount': 0, 'billCount': 0, 'contribu...",/akn/ie/debateRecord/joint_committee_on_foreig...,{'showAs': 'JOINT COMMITTEE ON FOREIGN AFFAIRS...
5,2007-12-19,"[{'debateSection': {'debateType': 'prelude', '...",2017-03-24T18:28:22+00:00,debate,"{'pdf': None, 'xml': {'uri': '/akn/ie/debateRe...",{'showAs': 'JOINT COMMITTEE ON CLIMATE CHANGE ...,"{'questionCount': 0, 'billCount': 0, 'contribu...",/akn/ie/debateRecord/joint_committee_on_climat...,{'showAs': 'JOINT COMMITTEE ON CLIMATE CHANGE ...
6,2007-12-19,"[{'debateSection': {'debateType': 'prelude', '...",2017-03-24T19:42:28+00:00,debate,"{'pdf': None, 'xml': {'uri': '/akn/ie/debateRe...","{'showAs': 'Seanad Éireann', 'uri': '/ie/oirea...","{'questionCount': 0, 'billCount': 3, 'contribu...",/akn/ie/debateRecord/seanad/2007-12-19/debate/...,"{'showAs': '23rd Seanad', 'committeeCode': '',..."
7,2007-12-18,"[{'debateSection': {'debateType': 'prelude', '...",2017-03-24T18:31:58+00:00,debate,"{'pdf': None, 'xml': {'uri': '/akn/ie/debateRe...",{'showAs': 'JOINT COMMITTEE ON THE ENVIRONMENT...,"{'questionCount': 0, 'billCount': 0, 'contribu...",/akn/ie/debateRecord/joint_committee_on_the_en...,{'showAs': 'JOINT COMMITTEE ON THE ENVIRONMENT...
8,2007-12-18,"[{'debateSection': {'debateType': 'prelude', '...",2017-03-24T18:31:04+00:00,debate,"{'pdf': None, 'xml': {'uri': '/akn/ie/debateRe...",{'showAs': 'JOINT COMMITTEE ON HEALTH AND CHIL...,"{'questionCount': 0, 'billCount': 0, 'contribu...",/akn/ie/debateRecord/joint_committee_on_health...,{'showAs': 'JOINT COMMITTEE ON HEALTH AND CHIL...
9,2007-12-18,"[{'debateSection': {'debateType': 'prelude', '...",2017-03-24T18:30:02+00:00,debate,"{'pdf': None, 'xml': {'uri': '/akn/ie/debateRe...",{'showAs': 'JOINT COMMITTEE ON EUROPEAN SCRUTI...,"{'questionCount': 0, 'billCount': 0, 'contribu...",/akn/ie/debateRecord/joint_committee_on_europe...,{'showAs': 'JOINT COMMITTEE ON EUROPEAN SCRUTI...


In [21]:
# Process all the files and stick them together in a single dataframe:
files = glob('data/year/*.json.xz')

data_frames = process_with_progress(debates_df, files)
df_raw = pd.concat(data_frames, axis=0, sort=True)

/home/xmg/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [25]:
flatten_columns = ['formats','chamber','counts', 'house'] # deal with debateSections later
df_raw = flatten_dataframe_columns(df_raw, flatten_columns)

In [26]:
df_raw.sample(5)

,formats.pdf,formats.xml.uri,formats.pdf.uri,chamber.showAs,chamber.uri,counts.questionCount,counts.billCount,counts.divisionCount,counts.contributorCount,counts.debateSectionCount,house.showAs,house.chamberType,house.committeeCode,house.houseCode,house.houseNo,house.uri,date,debateSections,debateType,lastUpdated,uri
286,NaN,/akn/ie/debateRecord/joint_committee_on_social...,NaN,JOINT COMMITTEE ON SOCIAL AND FAMILY AFFAIRS,/ie/oireachtas/committee,0,0,0,15,2,JOINT COMMITTEE ON SOCIAL AND FAMILY AFFAIRS,committee,joint_committee_on_social_and_family_affairs,dail,30,/ie/oireachtas/committee/dail/30/joint_committ...,2009-06-24,"[{'debateSection': {'debateType': 'prelude', '...",debate,2017-03-24T18:31:50+00:00,/akn/ie/debateRecord/joint_committee_on_social...
68,NaN,/akn/ie/debateRecord/seanad/1929-05-15/debate/...,NaN,Seanad Éireann,/ie/oireachtas/house/seanad,0,1,2,25,5,1928 Seanad,house,,seanad,1928,/ie/oireachtas/house/seanad/1928,1929-05-15,"[{'debateSection': {'debateType': 'prelude', '...",debate,2017-03-24T19:33:10+00:00,/akn/ie/debateRecord/seanad/1929-05-15/debate/...
62,NaN,/akn/ie/debateRecord/seanad/1941-05-08/debate/...,NaN,Seanad Éireann,/ie/oireachtas/house/seanad,0,0,0,4,4,3rd Seanad,house,,seanad,3,/ie/oireachtas/house/seanad/3,1941-05-08,"[{'debateSection': {'debateType': 'prelude', '...",debate,2017-03-24T19:34:04+00:00,/akn/ie/debateRecord/seanad/1941-05-08/debate/...
329,NaN,/akn/ie/debateRecord/standing_order_112_select...,/ie/oireachtas/debateRecord/standing_order_112...,Standing Order 112 Select Committee,/ie/oireachtas/committee,0,0,0,3,1,Standing Order 112 Select Committee,committee,standing_order_112_select_committee,dail,32,/ie/oireachtas/committee/dail/32/standing_orde...,2016-06-08,"[{'debateSection': {'debateType': 'debate', 'f...",debate,2017-10-02T10:20:37+01:00,/akn/ie/debateRecord/standing_order_112_select...
62,NaN,/akn/ie/debateRecord/dail/1963-05-30/debate/mu...,NaN,Dáil Éireann,/ie/oireachtas/house/dail,0,1,0,31,35,17th Dáil,house,,dail,17,/ie/oireachtas/house/dail/17,1963-05-30,"[{'debateSection': {'debateType': 'prelude', '...",debate,2017-03-24T19:11:58+00:00,/akn/ie/debateRecord/dail/1963-05-30/debate/main


In [27]:
df_raw.drop(['debateSections','formats.pdf'], axis=1, inplace=True)

In [28]:
describe_with_top_n(df_raw, n=50)

----------

# formats.xml.uri

###  Stats:

,formats.xml.uri
count,16560
unique,16559
top,/akn/ie/debateRecord/seanad/1936-01-01/debate/...
freq,2
missing,0


###  Top N:

...

# uri

###  Stats:

,uri
count,16560
unique,16559
top,/akn/ie/debateRecord/seanad/1936-01-01/debate/...
freq,2
missing,0


###  Top N:

,uri
/akn/ie/debateRecord/seanad/1936-01-01/debate/main,2
/akn/ie/debateRecord/dail/1952-11-12/debate/main,1
/akn/ie/debateRecord/dail/1971-02-17/debate/main,1
/akn/ie/debateRecord/joint_committee_on_arts_sport_tourism_community_rural_and_gaeltacht_affairs/2008-10-08/debate/main,1
/akn/ie/debateRecord/seanad/1993-04-08/debate/main,1
/akn/ie/debateRecord/dail/2008-10-29/debate/main,1
/akn/ie/debateRecord/joint_committee_on_justice_equality_defence_and_womens_rights/2009-01-27/debate/main,1
/akn/ie/debateRecord/select_committee_on_justice_equality_defence_and_womens_rights/2005-05-03/debate/main,1
/akn/ie/debateRecord/joint_committee_on_education_and_science/2009-12-02/debate/main,1
/akn/ie/debateRecord/joint_committee_on_european_union_affairs/2012-06-27/debate/main,1


In [29]:
df_raw.to_pickle('debates.p.xz', compression='xz') # gzip is faster, but xz makes much smaller files for git

In [30]:
import pandas_profiling

In [31]:
pandas_profiling.ProfileReport(df_raw)

/home/xmg/anaconda3/lib/python3.6/site-packages/pandas/plotting/_matplotlib/converter.py:102: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)
/home/xmg/anaconda3/lib/python3.6/site-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,20
Number of observations,16560
Total Missing (%),4.8%
Total size in memory,2.5 MiB
Average record size in memory,160.0 B
Numeric,6
Categorical,12
Boolean,0
Date,1
Text (Unique),0
Rejected,1
